### 문제 1-1 : 기본 체인 만들기 - AI 요리사

In [47]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [22]:
prompt = PromptTemplate.from_template('''
    당신은 어떤 재료로든 요리할 수 있는 일류 요리사입니다.
    다음 질문에 답해주세요.
    질문: {input}으로 요리할 수 있는 요리는 무엇이 있나요?(하나만 알려주세요) 그 요리의 레시피도 알려주세요.
''')

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

try:
    result = chain.invoke({"input": "계란, 밥, 김치"})
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

## **계란, 밥, 김치로 만들 수 있는 요리**

*   **김치볶음밥**

### **김치볶음밥 레시피**

**재료**

*   밥 2컵 
*   김치 1/2컵 
*   계란 1개 
*   참기름 1/2스푼 
*   고춧가루 1/2스푼 

**요리 방법**

1.  김치를 잘게 썰어주세요.
2.  팬에 참기름을 두르고 썰어 놓은 김치를 넣어 볶아주세요.
3.  김치가 볶아지면 고춧가루를 넣어 볶아주세요.
4.  김치에 밥을 넣어 볶아주세요.
5.  계란을 풀어서 밥에 넣어주세요.
6.  모든 재료를 섞어서 볶아주면 김치볶음밥이 완성됩니다.


### 문제 1-2 : 2단계 체인 만들기 - 영화 추천 시스템

In [81]:
# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_messages([
    "system", "당신은 영화 추천 전문가입니다. 사용자가 요청한 장르에 맞는 영화를 추천해주세요.",
    "human", "{genre} 장르에서 추천하는 영화를 한 편 알려주세요. 그 영화의 제목, 감독, 등장인물 등을 알려주세요."
    ])

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_messages([
    "system", "당신은 영화 줄거리 요약 전문가입니다. 사용자가 요청한 영화의 줄거리를 3줄로(줄바꿈해서) 요약해주세요.",
    "human","{movie} 영화의 줄거리를 알려주세요."
    ])

# OpenAI 모델 사용
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

try:
    chain1 = prompt1 | llm | StrOutputParser()
    response1 = chain1.invoke({"genre": "액션"})
    print("1단계 결과:\n", response1)

    chain2 = prompt2 | llm | StrOutputParser()
    response2 = chain2.invoke({"movie": response1})
    print("\n2단계 결과:\n", response2) 
except Exception as e:
    print(f"오류 발생: {e}")

1단계 결과:
 *   **영화 제목:** 존 윅 (John Wick)
*   **감독:** 채드 스태헬스키 (Chad Stahelski)
*   **등장인물:** 키아누 리브스 (Keanu Reeves), 마이클 얀 블랙 (Michael Nyqvist), 알피 앨런 (Alfie Allen), 윌리엄 더포 (Willem Dafoe) 
*   **영화 소개:** 전직 킬러 존 윅이 우연히 범죄 조직의 표적이 되어, 거대한 전쟁을 벌이는 내용을 그린 액션 영화입니다.

2단계 결과:
 은퇴한 킬러 존 윅은 사랑하는 아내를 암으로 잃고, 그녀가 남긴 마지막 선물인 '1969년 머스탱'을 타고 외로움을 달래던 중, 우연히 옛 동료 샘이자 현직 킬러인 이고르를 만나게 됩니다.

이고르의 부탁으로 존은 다시 킬러의 세계로 돌아오게 되지만, 이고르의 부탁을 거절하고 은퇴를 고수하려던 존은 이고르의 동료이자 러시아 마피아의 일원인 비고르가 운영하는 호텔 '콘티넨탈'에서 벌어지는 사건에 휘말리게 됩니다.

비고르의 조직과 전쟁을 벌이게 된 존 윅은 호텔의 규칙을 어기고, 비고르의 조직과 대립하며, 거대한 전쟁을 벌이게 됩니다.


### 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용

In [48]:
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "경기 의정부시에서 6중 추돌사고 낸 포르쉐 운전자가 마약 간이 검사에서 양성 반응을 보인 것으로 확인됐다.",
        "keywords": "의정부시, 추돌사고, 마약"
    }
]


example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])


# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."})
print(result.content)

키워드: 제미나이,구글, AI
